In [50]:
import numpy as np
import plotly.graph_objects as go

In [107]:
#Mettre un pas du maillage qui divise la distance entre val_max et val_min.

def creer_maillage(x_min : float, x_max : float, y_min : float, y_max : float, z_min : float, z_max : float, pas_du_maillage : float) -> tuple:
    """
    La fonction créer_maillage prend en entrée 6 float qui permettent de délémiter les bords du maillage et un pas.
    Le pas du maillage représente la distance séparant un point du prochain. 
    
    Elle renvoie un tuple de 3 arrays contenant eux-même N-array qui représentent toutes les combinaisons possibles
    discrétisant ainsi notre maillage en un  ensemble de points.
    """ 
    nb_points_entre_x_min_x_max = int((x_max - x_min) / (pas_du_maillage)) + 1
    nb_points_entre_y_min_y_max = int((y_max - y_min) / (pas_du_maillage)) + 1
    nb_points_entre_z_min_z_max = int((z_max - z_min) / (pas_du_maillage)) + 1

    axe_x = np.linspace(x_min,x_max,nb_points_entre_x_min_x_max)
    axe_y = np.linspace(y_min,y_max,nb_points_entre_y_min_y_max)
    axe_z = np.linspace(z_min,z_max,nb_points_entre_z_min_z_max)

    maillage = np.meshgrid(axe_x,axe_y,axe_z)

    return maillage

cote = 2
X,Y,Z = creer_maillage(-cote,cote,-cote,cote,-cote,cote,0.1)

print(f"Le maillage est composé de {len(X)*X[0].shape[0]*X[0].shape[1]:,} points.")

Le maillage est composé de 68,921 points.


In [52]:
def trouver_signe(x : float) -> str:
    if x >= 0:
        return '+'
    return '-'

In [121]:
def calculer_opacite_show_maillage(valeur : float, valeur_ref : float) -> float:
    if abs(valeur - valeur_ref) < 1:
        return 1
    if abs(valeur - valeur_ref) < 1.5:
        return 0.3
    if abs(valeur - valeur_ref) < 2.5:
        return 0.2
    else:
        return 0.1

In [54]:
def calcul_liste_arg_slider_show_maillage(valeur_ref : int, precision_slider : int) -> list:
    liste = [0,0]
    for valeur in range(precision_slider):
        liste.append(calculer_opacite_show_maillage(float(valeur) + 0.1, valeur_ref))
    return liste 

In [122]:
def show_maillage(X : np.array, Y : np.array, Z : np.array, afficher_plot : bool, afficher_array : bool, titre : str, grandeur = None, nom_slider  = None):
    """
    La fonction show_maillage() prend en argument les trois matrices issues de la fonction creer_maillage() et permet de
    visualiser soit les matrices, soit la représentation de celles-ci dans un plot.
    """
    
    precision_slider = 40 #plus le nombre est grand et plus la tolérance sur les valeurs diminuera. Il y aura moins de points affichés mais la précision sera plus grande. C'est un int.
    if type(precision_slider) != int:
        raise ValueError("La précision du slider doit être un int.")
    
    methode_affichage = 'browser' #doit être une string : "browser" ou "vscode"
    
    if afficher_plot:  
        if grandeur is not None:
            fig = go.Figure()

            mask_charge_positive = grandeur.flatten() > 0
            mask_charge_negative = grandeur.flatten() <= 0
        
            fig.add_trace(go.Scatter3d(
                x=X.flatten()[mask_charge_positive], y=Y.flatten()[mask_charge_positive], z=Z.flatten()[mask_charge_positive],
                mode="markers",
                marker=dict(size=2,
                            color=grandeur.flatten()[mask_charge_positive],
                            colorscale="inferno",
                            cmin = grandeur.min(),
                            cmax = grandeur.max(),
                            opacity=1,
                            colorbar=dict(
                            title=nom_slider,  
                            ticks="outside",  
                            tickvals=[grandeur.min(), grandeur.max()],  
                            ticktext=[f"{grandeur.min():{trouver_signe(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe(grandeur.max())}.2f}"],
                        )),
                showlegend = False
            ))

            fig.add_trace(go.Scatter3d(
                x=X.flatten()[mask_charge_negative], y=Y.flatten()[mask_charge_negative], z=Z.flatten()[mask_charge_negative],
                mode="markers",
                marker=dict(size=2,
                            color=grandeur.flatten()[mask_charge_negative],
                            colorscale="inferno",
                            cmin = grandeur.min(),
                            cmax = grandeur.max(),
                            opacity=1,
                            colorbar=dict(
                            title=nom_slider,  
                            ticks="outside",  
                            tickvals=[grandeur.min(), grandeur.max()],  
                            ticktext=[f"{grandeur.min():{trouver_signe(grandeur.min())}.2f}", f"{grandeur.max():{trouver_signe(grandeur.max())}.2f}"],  
                        )),
                showlegend= False
            ))
            
            tolerance = float((grandeur.max() - grandeur.min())/(2*precision_slider))
            for valeur in np.linspace(grandeur.min(),grandeur.max(),precision_slider):
                fig.add_trace(go.Scatter3d(
                x=X.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], y=Y.flatten()[abs(grandeur.flatten()-valeur) <= tolerance], z=Z.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
                mode="markers",
                marker=dict(size=2,
                            color=grandeur.flatten()[abs(grandeur.flatten()-valeur) <= tolerance],
                            colorscale="inferno",
                            cmin = grandeur.min(),
                            cmax = grandeur.max(),
                            opacity=1,
                            ),
                showlegend= False
            ))
            
            steps = []
            for valeur, charge in zip(range(precision_slider),np.linspace(grandeur.min(),grandeur.max(),precision_slider)):
                step = dict(
                    method='update',
                    args=[
                        {'marker.opacity': calcul_liste_arg_slider_show_maillage(valeur,precision_slider)},
                    ],
                    label = f" {charge:.2f}{chr(177)}{tolerance:.2f}"
                )
                steps.append(step)

            slider = [dict(
                active = int(precision_slider/2),
                currentvalue = {'prefix' : f'Valeur {nom_slider}  :'},
                pad = {'t':50},
                steps = steps
            )]
            
            liste_arg_reset_button = [1,1]
            for _ in range(precision_slider):
                liste_arg_reset_button.append(0)
            
            reset_button = dict(
                type='buttons',
                showactive=True,
                buttons=[
                    dict(label='Réinitialiser Opacité',
                         method='update',
                         args=[{'marker.opacity': liste_arg_reset_button}]
                    )
                ],
                x=0.1, 
                y=0.9,  
                xanchor="right",
                yanchor="bottom"
            )
            
            liste_arg_bouton_charge_positive = [1,0]
            for _ in range(precision_slider):
                liste_arg_bouton_charge_positive.append(0)
                
            bouton_charge_positive = dict(
                type='buttons',
                showactive=True,
                buttons=[
                    dict(label='Afficher charges +',
                         method='update',
                         args=[{'marker.opacity': liste_arg_bouton_charge_positive}]
                    )
                ],
                x=0.1, 
                y=0.7,  
                xanchor="right",
                yanchor="bottom"
            )
            
            liste_arg_bouton_charge_negative = [0,1]
            for _ in range(precision_slider):
                liste_arg_bouton_charge_negative.append(0)
                
            bouton_charge_negative = dict(
                type='buttons',
                showactive=True,
                buttons=[
                    dict(label='Afficher charges -',
                         method='update',
                         args=[{'marker.opacity': liste_arg_bouton_charge_negative}]
                    )
                ],
                x=0.1, 
                y=0.5,  
                xanchor="right",
                yanchor="bottom"
            )

            fig.update_layout(
                title = titre,
                sliders = slider,
                updatemenus = [reset_button,bouton_charge_positive,bouton_charge_negative],
                scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
            )
            fig.show(methode_affichage)

        else:
            fig = go.Figure(data=[go.Scatter3d(
                x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
                mode="markers",
                marker=dict(size=2, opacity=0.8) 
            )])

            fig.update_layout(
                title=titre,
                scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
            )
            fig.show(methode_affichage)

    if afficher_array:
        print(f"La première matrice ressemble à :\n\n{X},\n\n"
              f"La deuxième matrice ressemble à :\n\n{Y},\n\n"
              f"La dernière matrice ressemble à :\n\n{Z}")
        
    return None

show_maillage(X, Y, Z, False, False, "Visualisation du maillage")

In [56]:
#Il faut encore gérer le cas X == Y == Z

def find_valeurs_point(x : float, y : float, z : float, X : np.array, Y : np.array, Z : np.array) -> tuple:
    """
    Cette fonction permet en prenant 3 matrices X,Y,Z issues de la fonction creer_maillage() en attribut 
    et 3 points correspondant au point de l'espace considéré dont on veut connaître les valeurs associées à X,Y,Z.
    Si les trois matrices sont égales, alors elle renvoie la valeur associée au point considéré.
    """
    nb_points_x = np.shape(X[0])[0]
    nb_points_y = len(X)
    nb_points_z = np.shape(X[0])[1]
    
    val_min_x = X[0][0,0]
    val_max_x = X[0][nb_points_x-1,0]
    val_min_y = Y[0][0,0]
    val_max_y = Y[nb_points_y-1][0,0]
    val_min_z = Z[0][0,0]
    val_max_z = Z[0][0,nb_points_z-1]

    if (nb_points_x-1) != 0 and (nb_points_y-1) != 0 and (nb_points_z-1) != 0:
        pas_suivant_x = float((int(val_max_x - val_min_x))  / (nb_points_x - 1))
        pas_suivant_y = float((int(val_max_y - val_min_y))  / (nb_points_y - 1))
        pas_suivant_z = float((int(val_max_z - val_min_z))  / (nb_points_z - 1))
    else:
        raise ZeroDivisionError()
    
    val_indice_x = int((x - val_min_x) / pas_suivant_x)
    val_indice_y = int((y - val_min_y) / pas_suivant_y)
    val_indice_z = int((z - val_min_z) / pas_suivant_z)

    return float(X[val_indice_y][val_indice_x,val_indice_z]), float(Y[val_indice_y][val_indice_x,val_indice_z]), float(Z[val_indice_y][val_indice_x,val_indice_z])

print(find_valeurs_point(0,1,-1,X,Y,Z))



(0.0, 1.0, -1.0)


In [112]:
def calcul_charge_tout_point_maillage(X : np.array, Y : np.array, Z : np.array, fonction) -> tuple:
    """
    La fonction calcul_charge_tout_point_maillage() prend en entrée les 3 matrices qu'output la fonction
    creer_maillage() afin de calculer la charge portée par tous les points selon la fonction fournie.
    """
    return fonction(X,Y,Z)

maillage_charge = calcul_charge_tout_point_maillage(X,Y,Z, lambda x,y,z : np.exp(- (x**2 + y**2 + z**2)))


In [132]:
show_maillage(X,Y,Z,True,False,"Visualisation du maillage chargé", maillage_charge, "charge (C)")

In [59]:
def placer_charge(x : float, y : float, z : float, liste_charge_de_la_distribution : list) -> list:
    """
    Elle permet d'ajouter un np.array de coordonnées, représentant le fait qu'une charge y est placée, dans une liste décrivant la distribution
    """
    liste_charge_de_la_distribution.append(np.array([x,y,z], float))
    return liste_charge_de_la_distribution

test = list()

for i in range(2):
    for j in range(2):
        for k in range(2):
            placer_charge(i,j,k,test)

print(test)

[array([0., 0., 0.]), array([0., 0., 1.]), array([0., 1., 0.]), array([0., 1., 1.]), array([1., 0., 0.]), array([1., 0., 1.]), array([1., 1., 0.]), array([1., 1., 1.])]


In [60]:
test = list()

for i in range(2):
    for j in range(2):
        for k in range(2):
            placer_charge(i,j,k,test)

def deplacer_charge(x : float, y : float, z : float, dx : float, dy : float, dz : float, liste_charge_de_la_distribution) -> list:
    """
    La fonction prend les coordonnées du point (x,y,z) dans la liste_charge_de_la_distribution et les déplace respectivement de (dx,dy,dz).
    """
    for array in liste_charge_de_la_distribution:
        if np.array_equal(np.array([x,y,z],float),array):
            array[0] += dx
            array[1] += dy
            array[2] += dz

    return liste_charge_de_la_distribution

print(deplacer_charge(0,1,0,0.5,2,1,test))

[array([0., 0., 0.]), array([0., 0., 1.]), array([0.5, 3. , 1. ]), array([0., 1., 1.]), array([1., 0., 0.]), array([1., 0., 1.]), array([1., 1., 0.]), array([1., 1., 1.])]
